In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
df = pd.read_csv("/kaggle/input/baris/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv")

In [ ]:
#df = df.sample(frac=0.001,random_state=42) 
#df = df.sample(n=50000, random_state=42)
#df = pd.read_csv('mta.csv', nrows=5000)
#df = pd.read_csv('mta13.csv',parse_dates=['transit_timestamp'], low_memory=True)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df = df.drop('payment_method', axis=1)
df = df.drop('station_complex_id', axis =1)
df = df.drop('Georeference', axis=1)

In [ ]:
df = df.sample(frac=0.001,random_state=42)  


In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'])

In [ ]:
df['transit_timestamp'].max()

In [ ]:
df['transit_timestamp'].min()

In [ ]:
# Set the start and end dates
start_date = '2022-05-30 00:00:00'
end_date = '2023-05-30 00:00:00'

# Filter the data between the start and end dates
mask = (df['transit_timestamp'] >= start_date) & (df['transit_timestamp'] < end_date)
filtered_df = df.loc[mask]

print(filtered_df)

df= filtered_df
df.info()

In [ ]:
df['hour'] = df['transit_timestamp'].dt.hour

In [ ]:
df['day_of_week'] = df['transit_timestamp'].dt.dayofweek

In [ ]:
df['month'] = df['transit_timestamp'].dt.month

In [ ]:
# Remove commas from 'ridership' column

df['ridership'] = df['ridership'].astype(str) 
df['ridership'] = df['ridership'].str.replace(',', '')

In [ ]:
df['ridership'] = df['ridership'].astype(int)

In [ ]:
df['transfers'] = df['transfers'].astype(str) 
df['transfers'] = df['transfers'].str.replace(',', '')

In [ ]:
df['transfers'] = df['transfers'].astype(int)

In [ ]:
df.info()

In [ ]:
!pip install --upgrade numpy==1.17.2
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
borough_ridership = df.groupby('borough')['ridership'].sum()

plt.figure(figsize=(10,6))
borough_ridership.plot(kind='bar', color='orange', alpha=0.7)
plt.title('Total number of passengers by Boroughs between 2022–05-30 and 2023–05–30')
plt.xlabel('Borough')
plt.ylabel('Total Ridership -Milion')
plt.show()


In [ ]:
borough_transfers = df.groupby('borough')['transfers'].sum()

plt.figure(figsize=(10,6))
borough_transfers.plot(kind='bar', color='green', alpha=0.6)
plt.title('Total number of transfers by Boroughs between 2022–05–30 and 2023–05–30')
plt.xlabel('Borough')
plt.ylabel('Total Transfers')
plt.show()

In [ ]:
import matplotlib.pyplot as plt 

boroughs = df['borough'].unique() 
borough_monthly_ridership = df.groupby(['borough', 'month'])['ridership'].mean().unstack(level=0)


plt.figure(figsize=(10,6)) 

for borough in boroughs: 
  borough_monthly_ridership[borough].plot(label=borough)

plt.title('Ridership by Months') 
plt.xlabel('Month') 
plt.ylabel('Ridership') 
plt.legend() 
plt.show()

In [ ]:
borough_hourly_ridership = df.groupby(['borough', 'hour'])['ridership'].mean().unstack(level=0)

boroughs = df['borough'].unique()

plt.figure(figsize=(14,8))

for borough in boroughs:
    borough_hourly_ridership[borough].plot(label=borough)

plt.title('Ridership by Borough in 24 Hours')
plt.xlabel('Hours')
plt.ylabel('Number of Ridership')
plt.legend()
plt.show()


In [ ]:
import plotly.graph_objects as go

borough_hourly_ridership = df.groupby(['borough', 'hour'])['ridership'].mean().unstack(level=0)
boroughs = df['borough'].unique()

fig = go.Figure()

for borough in boroughs:
    fig.add_trace(go.Scatter(x=borough_hourly_ridership.index, 
                             y=borough_hourly_ridership[borough], 
                             mode='lines+markers',
                             name=borough))

fig.update_layout(title='Ridership by Borough in 24 Hours',
                   xaxis_title='Hours',
                   yaxis_title='Number of Ridership')

fig.show()


In [ ]:
pivot_table = df.pivot_table(values='ridership', index='hour', columns='day_of_week', aggfunc='mean') 
sns.heatmap(pivot_table, cmap='coolwarm') 
plt.title('Heatmap of Average Ridership by Hour and Day of Week') 
plt.show()


In [ ]:
df.groupby(df['transit_timestamp'].dt.month)['ridership'].mean().plot() 
plt.title('Average Ridership by Month') 
plt.xlabel('Month') 
plt.ylabel('Average Ridership') 
plt.show()

In [ ]:
import plotly.graph_objs as go

avg_ridership = df.groupby(df['transit_timestamp'].dt.month)['ridership'].mean()

fig = go.Figure(data=go.Scatter(x=avg_ridership.index, y=avg_ridership.values, line=dict(color='orange')))



fig.update_layout(title='Average Ridership by Month',
                   xaxis_title='Month',
                   yaxis_title='Average Ridership')

fig.show()




In [ ]:
subset = df[['ridership', 'hour', 'day_of_week', 'month']]
correlation_matrix = subset.corr()
print(correlation_matrix)

In [ ]:
import seaborn as sns

sns.heatmap(correlation_matrix, annot=True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(df['longitude'], df['latitude'])
plt.title('Geospatial Distribution of Station Complexes')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.show()

In [ ]:
!pip install folium

In [ ]:
import folium
# Create a map centered around the average coordinates
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=13)
# Add a marker for every record in the filtered data
for each in df.iterrows():
    folium.Marker([each[1]['latitude'], each[1]['longitude']],
                  popup=each[1]['station_complex']).add_to(m)
# Show the map
m

In [ ]:
import folium
# Calculate total ridership for each station
station_ridership = df.groupby('station_complex')['ridership'].sum().reset_index()
# Normalize ridership for better visualization (you can skip this if not needed)
station_ridership['ridership'] = (station_ridership['ridership'] - station_ridership['ridership'].min()) / \
                                  (station_ridership['ridership'].max() - station_ridership['ridership'].min())
# Merge ridership data back to original dataframe (to get coordinates for each station)
df_merged = df.merge(station_ridership, on='station_complex', how='left')
# Create map centered around the average coordinates
m = folium.Map(location=[df_merged['latitude'].mean(), df_merged['longitude'].mean()], zoom_start=13)
# Add a circle marker for every record in the filtered data, use ridership_y as the ridership data
for each in df_merged.drop_duplicates(subset='station_complex').iterrows():
    folium.CircleMarker(
        location=[each[1]['latitude'], each[1]['longitude']],
        radius=each[1]['ridership_y']*10,  # change this value to adjust the size of circles
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=each[1]['station_complex']
    ).add_to(m)
# Show the map
m

In [ ]:
import folium
# Calculate total ridership for each station
station_ridership = df.groupby('station_complex')['ridership'].sum().reset_index()
# Normalize ridership for better visualization (you can skip this if not needed)
station_ridership['ridership'] = (station_ridership['ridership'] - station_ridership['ridership'].min()) / \
                                  (station_ridership['ridership'].max() - station_ridership['ridership'].min())
# Merge ridership data back to original dataframe (to get coordinates for each station)
df_merged = df.merge(station_ridership, on='station_complex', how='left')
# Create map centered around the average coordinates
m = folium.Map(location=[df_merged['latitude'].mean(), df_merged['longitude'].mean()], zoom_start=13)
# Add a circle marker for every record in the filtered data, use ridership_y as the ridership data
for each in df_merged.drop_duplicates(subset='station_complex').iterrows():
    folium.CircleMarker(
        location=[each[1]['latitude'], each[1]['longitude']],
        radius=each[1]['ridership_y']*10,  # change this value to adjust the size of circles
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=each[1]['station_complex']
    ).add_to(m)
# Show the map
m

In [ ]:
import folium
from folium.plugins import HeatMap 

# Create a new map centered around your area of interest
m = folium.Map(location=[40.7128, -74.0060], zoom_start=11) 

# Generate the heat map
station_arr = df[['latitude', 'longitude', 'ridership']].values  # Convert DataFrame to NumPy array
HeatMap(station_arr, radius=15).add_to(m) 

# Show the map
m

In [ ]:
!pip install --upgrade pip
!pip install HeatMap

In [ ]:
import folium
from folium.plugins import HeatMap

# Create base map
m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

# Add a circle marker for each station with gradient color based on ridership
for index, row in df.iterrows():
    # Define color function
    def color(ridership):
        if ridership in range(0,1000):
            col = 'green'
        elif ridership in range(1001,5000):
            col = 'blue'
        else:
            col='red'
        return col 

    folium.CircleMarker(
        location=(row['latitude'], row['longitude']),
        radius=5,  # Adjust size as needed
        color=color(row['ridership']),  # Use color function
        fill=True,
        fill_opacity=0.7,
        popup=f'Station: {row["station_complex"]}\nRidership count: {row["ridership"]}'
    ).add_to(m)

# Adding a heat map layer to the map can help visualize the concentration of ridership
station_arr = df[['latitude', 'longitude', 'ridership']].values  # Convert DataFrame to NumPy array
HeatMap(station_arr, radius=15).add_to(m)

m


In [ ]:
import folium
from folium.plugins import HeatMap

# Create a new map centered around your area of interest
m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

# Generate the heat map
station_arr = df[['latitude', 'longitude', 'ridership']].values  # Convert DataFrame to NumPy array
HeatMap(station_arr, radius=15).add_to(m)

# Show the map
m


In [ ]:
!pip install plotly


In [ ]:
df.info()

In [ ]:
import plotly.express as px 

fig = px.scatter_3d(df, x='ridership', y='transfers', z='hour', color='borough')
fig.show()

In [ ]:
import plotly.express as px 

fig = px.scatter(df, x="hour", y="ridership", color="borough", title="Ridership by Hours")
fig.show()

In [ ]:
import plotly.express as px 

fig = px.scatter(df, x="hour", y="transfers", color="borough", title="Ridership by Hours")
fig.show()

In [ ]:
fig = px.box(df, x="borough", y="ridership", title="Ridership by Boroughs")
fig.show()


In [ ]:
df_grouped = df.groupby(['borough', 'hour'])['ridership'].mean().reset_index()
fig = px.line(df_grouped, x="hour", y="ridership", color="borough", title="Average Ridership by Hours for Each Borough")
fig.show()


In [ ]:
#Bar plot of total ridership by borough:

df_grouped = df.groupby('borough')['ridership'].mean().reset_index()

fig = px.bar(df_grouped, x="borough", y="ridership", title="Total Ridership by Borough")
fig.show()

In [ ]:
#Interactive 3D Scatter Plot of Ridership, Transfers and Hours
#If your data has more than two numeric variables, you might consider a 3D scatter plot which allows you to visualize three numeric variables at once.

import plotly.express as px

fig = px.scatter_3d(df, x='ridership', y='transfers', z='hour', color='borough')
fig.show()

In [ ]:
#Sunburst Chart of Ridership by Borough and Stations
#A sunburst chart is used to visualize hierarchical data, spanned across multiple categories.

import plotly.express as px

df_sunburst = df.groupby(['borough', 'station_complex'])['ridership'].sum().reset_index()

fig = px.sunburst(df_sunburst, path=['borough', 'station_complex'], values='ridership', color='borough', title='Ridership by Borough and Station')
fig.show()
fig.write_image("fig1.jpeg")

In [ ]:
#Density Heatmap of Ridership by Hour and Day of Week
#A density heatmap can help understand the concentration and distribution of ridership across different times of the week.

import plotly.express as px

# For this plot, we need to make sure 'day_of_week' is in the correct format.
df['day_of_week'] = df['transit_timestamp'].dt.dayofweek

fig = px.density_heatmap(df, x='hour', y='day_of_week', z='ridership', nbinsx=24, nbinsy=7)
fig.show()

In [ ]:
# First, create a dictionary mapping the day of the week integer to its corresponding weekday name.
day_dict = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}

# Next, use the map() function to apply the dictionary to the 'day_of_week' column.
df['day_of_week'] = df['day_of_week'].map(day_dict)

# Convert the 'day_of_week' column to a categorical type and specify the order.
df['day_of_week'] = pd.Categorical(df['day_of_week'], categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)

import plotly.express as px
fig = px.density_heatmap(df, x='hour', y='day_of_week', z='ridership', nbinsx=24, nbinsy=7)
fig.show()



In [ ]:
import plotly.express as px 

# Map the 'day_of_week' numbers to day names
day_dict = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
df['day_of_week'] = df['transit_timestamp'].dt.dayofweek.map(day_dict)

# Create the heatmap
fig = px.density_heatmap(df, x='hour', y='day_of_week', z='ridership', nbinsx=24, nbinsy=7)
fig.show()


In [ ]:
fig.savefig()

In [ ]:
!pip install cufflinks

In [ ]:
import cufflinks as cf
cf.set_config_file(theme='pearl')

In [ ]:
print("Cufflinks Version : {}".format(cf.__version__))

In [ ]:
print("List of Cufflinks Themes : ", cf.getThemes())

## Setting Pearl Theme
cf.set_config_file(theme='pearl', sharing='public', offline=True) # dimensions=(), margin=(20,20,20,20)

In [ ]:
fig = df.figure(kind="scatter",
               x="borough", y='ridership',
               mode='markers',categories="borough",
               theme="white",
               xTitle="Borughs", yTitle="Number of Ridership",
               title="Ridership by Borough")

fig

In [ ]:
import plotly.express as px
import plotly.io as pio



# Bar Chart of Average Ridership by Station
fig = px.bar(df.groupby("station_complex")["ridership"].mean().reset_index(), x="station_complex", y="ridership")
fig.show()


# Line Plot of Ridership Over Time
fig = px.line(df.sort_values("transit_timestamp"), x="transit_timestamp", y="ridership")
fig.show()


# Box Plot of Ridership by Hour
fig = px.box(df, x="hour", y="ridership")
fig.show()


# Heatmap of Ridership by Day of Week and Hour
fig = px.density_heatmap(df, x="hour", y="day_of_week", z="ridership", nbinsx=24, nbinsy=7)
fig.show()




# Histogram of Transfers
fig = px.histogram(df, x="transfers")
fig.show()

# Pie Chart of Ridership by Borough
fig = px.pie(df.groupby("borough")["ridership"].sum().reset_index(), values='ridership', names='borough')
fig.show()


# Geographical Scatter Plot of Stations
fig = px.scatter_geo(df, lat="latitude", lon="longitude", color="ridership", size="transfers")
fig.show()

# Violin Plot of Ridership by Month
fig = px.violin(df, x="month", y="ridership")
fig.show()

# Area Chart of Transfers Over Time
fig = px.area(df.sort_values("transit_timestamp"), x="transit_timestamp", y="transfers")
fig.show()

# Bubble Chart of Stations by Latitude and Longitude, colored by Ridership, sized by Transfers
fig = px.scatter(df, x="longitude", y="latitude", color="ridership", size="transfers", hover_name="station_complex", title="Stations by Location, Ridership, and Transfers")
fig.show()

# Sunburst Chart of Ridership by Borough and Station
fig = px.sunburst(df, path=['borough', 'station_complex'], values='ridership')
fig.show()